individual model evaluation

In [23]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

In [24]:
# Import pandas libraries
import pandas as pd

# Load the dataset
# data = pd.read_csv("Data/activemq_result.csv", delimiter=',')
data = pd.read_csv("Data/camel_result.csv", delimiter=',')


# Explore the dataset
print(data.head()) 



                                         Method name  C20  C3  C4  C1  C5  C6  \
0  org.apache.camel.api.management.mbean.CamelOpe...    1   4   0   4   2   2   
1  org.apache.camel.api.management.mbean.CamelOpe...    3   7   0   7   1   5   
2  org.apache.camel.api.management.mbean.CamelOpe...    4   6   0   6   1   4   
3  org.apache.camel.api.management.mbean.CamelOpe...    1   4   0   4   2   2   
4  org.apache.camel.api.management.mbean.CamelOpe...    6   6   0   6   1   4   

   C2  C21  C18  ...  H4  H1  H2  H3       H12  H13  H14  H15  H5  \
0   0    3    1  ...   4   4   0   4  1.000000  0.0  0.0  1.0   1   
1   0    2    1  ...   8   8   0   8  1.142857  0.0  0.0  1.0   1   
2   0    2    1  ...   2   1   0   1  0.166667  0.0  0.0  0.5   1   
3   0    3    1  ...   1   1   0   1  0.250000  0.0  0.0  1.0   1   
4   0    2    1  ...   3   3   0   3  0.500000  0.0  0.0  1.0   1   

       bug-prone  
0  not bug-prone  
1  not bug-prone  
2  not bug-prone  
3  not bug-prone  
4  

In [25]:
# copy data
data_transform = data.copy()

In [26]:

# Convert 'bug-prone' column to 0 and 1
data_transform['bug-prone'] = data_transform['bug-prone'].apply(lambda x: 1 if x.strip() == 'bug-prone' else 0)

# Display the updated DataFrame
print(data_transform.head()) 


                                         Method name  C20  C3  C4  C1  C5  C6  \
0  org.apache.camel.api.management.mbean.CamelOpe...    1   4   0   4   2   2   
1  org.apache.camel.api.management.mbean.CamelOpe...    3   7   0   7   1   5   
2  org.apache.camel.api.management.mbean.CamelOpe...    4   6   0   6   1   4   
3  org.apache.camel.api.management.mbean.CamelOpe...    1   4   0   4   2   2   
4  org.apache.camel.api.management.mbean.CamelOpe...    6   6   0   6   1   4   

   C2  C21  C18  ...  H4  H1  H2  H3       H12  H13  H14  H15  H5  bug-prone  
0   0    3    1  ...   4   4   0   4  1.000000  0.0  0.0  1.0   1          0  
1   0    2    1  ...   8   8   0   8  1.142857  0.0  0.0  1.0   1          0  
2   0    2    1  ...   2   1   0   1  0.166667  0.0  0.0  0.5   1          0  
3   0    3    1  ...   1   1   0   1  0.250000  0.0  0.0  1.0   1          0  
4   0    2    1  ...   3   3   0   3  0.500000  0.0  0.0  1.0   1          0  

[5 rows x 42 columns]


In [27]:
print(data_transform.shape)

(12081, 42)


In [28]:

# split feature data and target data
feature_X = data_transform.drop(columns=['Method name','bug-prone'])
y = data_transform['bug-prone']



In [29]:
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
# smote = SMOTE(random_state=42)

# Apply SMOTETomek first (on original feature space)
smote = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote.fit_resample(feature_X, y)

# Apply StandardScaler after resampling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)



In [30]:
X = X_scaled
y = y_resampled

In [31]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import (
    make_scorer, precision_score, recall_score, f1_score, 
    matthews_corrcoef, roc_auc_score
)

# Define custom scorers for cross-validation
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='binary'),  # For binary classification
    'recall': make_scorer(recall_score, average='binary'),
    'f1': make_scorer(f1_score, average='binary'),
    'mcc': make_scorer(matthews_corrcoef),
    'auc': make_scorer(roc_auc_score) # , needs_proba=True
}

rf_classifier = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=5, 
                                       min_samples_leaf=2, class_weight='balanced', random_state=42)
ada_classifier = AdaBoostClassifier(n_estimators=100, learning_rate=0.8, random_state=42)
bagging_classifier = BaggingClassifier(n_estimators=100, max_samples=0.8, random_state=42)
knn_classifier = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')
mlp_classifier = MLPClassifier(activation='relu', hidden_layer_sizes=(200,100), max_iter=2000, 
                            learning_rate='adaptive', random_state=42)
gb_classifier = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42)
hgb_classifier = HistGradientBoostingClassifier(random_state=42)  
dt_classifier = DecisionTreeClassifier(random_state=42)  # max_depth=10, min_samples_split=10, min_samples_leaf=5,
svm_classifier = SVC(random_state=42, probability=True, C=10, kernel='poly', gamma='scale')
gnb_classifier = GaussianNB(var_smoothing=1e-9)
lr_classifier = LogisticRegression(class_weight='balanced', random_state=42)
xgb_classifier = XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42)
cb_classifier = CatBoostClassifier(iterations=200, learning_rate=0.05, depth=4, verbose=0, random_state=42)



In [32]:

# Helper function to display results
def print_cv_results(results):
    for metric in scoring.keys():
        mean = results[f'test_{metric}'].mean()
        std = results[f'test_{metric}'].std()
        print(f"{metric.capitalize()}: {mean:.4f} ± {std:.4f}")

def crossvalidate_fun(classifier, X_train, y_train):
    cv_results = cross_validate(classifier, X_train, y_train, cv=10, scoring=scoring)
    print_cv_results(cv_results)



In [33]:
# crossvalidate_fun(rf_classifier,X, y)


In [34]:
# crossvalidate_fun(ada_classifier,X, y)


In [35]:
# crossvalidate_fun(bagging_classifier,X, y)


In [36]:
# crossvalidate_fun(knn_classifier,X, y)


In [37]:
# crossvalidate_fun(mlp_classifier,X, y)


In [38]:
# crossvalidate_fun(gb_classifier,X, y)


In [39]:
# crossvalidate_fun(hgb_classifier,X, y)


In [40]:
# crossvalidate_fun(dt_classifier,X, y)


Accuracy: 0.8650 ± 0.0612
Precision: 0.8549 ± 0.0380
Recall: 0.8767 ± 0.1113
F1: 0.8632 ± 0.0726
Mcc: 0.7340 ± 0.1196
Auc: 0.8650 ± 0.0612


In [41]:
# crossvalidate_fun(svm_classifier,X, y)


Accuracy: 0.7794 ± 0.0426
Precision: 0.7417 ± 0.0408
Recall: 0.8592 ± 0.0426
F1: 0.7958 ± 0.0387
Mcc: 0.5665 ± 0.0850
Auc: 0.7794 ± 0.0426


In [42]:
# crossvalidate_fun(gnb_classifier,X, y)


Accuracy: 0.6324 ± 0.0180
Precision: 0.8266 ± 0.0551
Recall: 0.3374 ± 0.0242
F1: 0.4783 ± 0.0268
Mcc: 0.3293 ± 0.0470
Auc: 0.6324 ± 0.0180


In [43]:
crossvalidate_fun(lr_classifier,X, y)


c:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Accuracy: 0.8025 ± 0.0502
Precision: 0.8486 ± 0.0413
Recall: 0.7338 ± 0.0794
F1: 0.7860 ± 0.0633
Mcc: 0.6111 ± 0.0966
Auc: 0.8025 ± 0.0502


c:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
# crossvalidate_fun(xgb_classifier,X, y)


Accuracy: 0.8596 ± 0.0681
Precision: 0.8716 ± 0.0318
Recall: 0.8392 ± 0.1293
F1: 0.8513 ± 0.0869
Mcc: 0.7235 ± 0.1284
Auc: 0.8596 ± 0.0681


In [45]:
# crossvalidate_fun(cb_classifier,X, y)

Accuracy: 0.8547 ± 0.0661
Precision: 0.8695 ± 0.0370
Recall: 0.8306 ± 0.1198
F1: 0.8466 ± 0.0828
Mcc: 0.7130 ± 0.1257
Auc: 0.8547 ± 0.0661
